In [8]:
import pyvisa 

rm = pyvisa.ResourceManager('@py')
rm.list_resources()

()

In [1]:
import numpy as np
import random 
class SimulatedKeithley6517B:
    def __init__(self, mode='resistor', resistance=1e3, diode_is=1e-9, diode_n=1, temperature=300):
        """
        Initialize the simulated instrument.
        :param mode: 'resistor' or 'diode'
        :param resistance: Resistance value in Ohms (for resistor mode)
        :param diode_is: Saturation current (for diode mode)
        :param diode_n: Ideality factor (for diode mode)
        :param temperature: Temperature in Kelvin (for diode mode)
        """
        self.mode = mode
        self.resistance = resistance  # Ohms, used in resistor mode
        self.diode_is = diode_is      # Saturation current, used in diode mode
        self.diode_n = diode_n        # Ideality factor, used in diode mode
        self.temperature = temperature  # Temperature in Kelvin for diode mode
        self.voltage = 0.0  # Voltage applied
        self.current = 0.0  # Current measured
        self.noise = 0.01

    def set_voltage(self, voltage):
        """
        Sets the applied voltage.
        :param voltage: Voltage in Volts
        """
        self.voltage = voltage
        self._update_current()

    def measure_current(self):
        """
        Returns the current based on the applied voltage and component type (resistor/diode).
        :return: Measured current in Amps
        """
        return self.current * (1 + self.noise * random.random())

    def _update_current(self):
        """
        Updates the current based on the applied voltage and the component type.
        """
        if self.mode == 'resistor':
            # Ohm's law for resistor: I = V / R
            self.current = self.voltage / self.resistance
        elif self.mode == 'diode':
            # Shockley diode equation: I = I_s * (exp(V / (n * V_T)) - 1)
            V_T = 8.617333262e-5 * self.temperature  # Thermal voltage in Volts
            self.current = self.diode_is * (np.exp(self.voltage / (self.diode_n * V_T)) - 1)

    def get_voltage(self):
        """
        Returns the applied voltage.
        :return: Voltage in Volts
        """
        return self.voltage

    def configure_current_range(self, current_range):
        """
        Simulate configuring the current range (stub function for compatibility).
        :param current_range: Current range value
        """
        pass

    def configure_voltage_source(self, voltage):
        """
        Simulate configuring the voltage source (stub function for compatibility).
        :param voltage: Voltage value
        """
        self.set_voltage(voltage)

    def enable_output(self):
        """
        Simulate enabling the output (stub function for compatibility).
        """
        pass

    def disable_output(self):
        """
        Simulate disabling the output (stub function for compatibility).
        """
        self.set_voltage(0)
        self.current = 0.0


In [2]:
import sys
import numpy as np
import csv
import pyqtgraph as pg
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QDoubleSpinBox, QSpinBox, QPushButton, QFileDialog)
from PyQt5.QtCore import QTimer, QElapsedTimer

class MeasurementSimulator(QWidget):
    def __init__(self):
        super().__init__()

        # Initialize simulation variables
        self.simulated_device = SimulatedKeithley6517B(mode='diode', resistance=1e3)
        self.voltage_min = 0
        self.voltage_max = 1
        self.compliance_current = 1e-3
        self.voltage_step = 0.1
        self.collection_time = 1000  # in milliseconds
        self.measurements = []

        # Setup GUI components
        self.initUI()

        # Setup timer for measurements
        self.timer = QTimer()
        self.timer.timeout.connect(self.perform_measurement)

        # Timer for signal integration
        self.elapsed_timer = QElapsedTimer()

    def initUI(self):
        self.setWindowTitle('Simulated Keithley 6517B')
        layout = QVBoxLayout()

        # Voltage range input
        voltage_range_layout = QHBoxLayout()
        self.voltage_min_input = QDoubleSpinBox()
        self.voltage_min_input.setRange(-10, 10)
        self.voltage_min_input.setValue(0)
        self.voltage_max_input = QDoubleSpinBox()
        self.voltage_max_input.setRange(-10, 10)
        self.voltage_max_input.setValue(1)
        voltage_range_layout.addWidget(QLabel('Vmin (V):'))
        voltage_range_layout.addWidget(self.voltage_min_input)
        voltage_range_layout.addWidget(QLabel('Vmax (V):'))
        voltage_range_layout.addWidget(self.voltage_max_input)
        layout.addLayout(voltage_range_layout)

        # Voltage step input
        voltage_step_layout = QHBoxLayout()
        self.voltage_step_input = QDoubleSpinBox()
        self.voltage_step_input.setRange(0.001, 1)
        self.voltage_step_input.setValue(0.1)
        voltage_step_layout.addWidget(QLabel('Voltage step (V):'))
        voltage_step_layout.addWidget(self.voltage_step_input)
        layout.addLayout(voltage_step_layout)

        # Compliance current input
        compliance_layout = QHBoxLayout()
        self.compliance_input = QDoubleSpinBox()
        self.compliance_input.setRange(1e-9, 1)
        self.compliance_input.setValue(1e-3)
        compliance_layout.addWidget(QLabel('Compliance current (A):'))
        compliance_layout.addWidget(self.compliance_input)
        layout.addLayout(compliance_layout)

        # Collection time input
        collection_time_layout = QHBoxLayout()
        self.collection_time_input = QSpinBox()
        self.collection_time_input.setRange(10, 5000)  # Time in ms
        self.collection_time_input.setValue(1000)
        collection_time_layout.addWidget(QLabel('Collection time (ms):'))
        collection_time_layout.addWidget(self.collection_time_input)
        layout.addLayout(collection_time_layout)

        # Start/Stop button
        button_layout = QHBoxLayout()
        self.start_button = QPushButton('Start Measurement')
        self.start_button.clicked.connect(self.start_measurement)
        self.stop_button = QPushButton('Stop Measurement')
        self.stop_button.clicked.connect(self.stop_measurement)
        button_layout.addWidget(self.start_button)
        button_layout.addWidget(self.stop_button)
        layout.addLayout(button_layout)

        # Plot area for I(V) and abs(I(V)) using PyQtGraph
        self.plot_widget = pg.GraphicsLayoutWidget()
        self.iv_plot = self.plot_widget.addPlot(title="I(V)")
        self.abs_iv_plot = self.plot_widget.addPlot(title="|I(V)|")
        layout.addWidget(self.plot_widget)

        self.abs_iv_plot.setLogMode(False, True)  # Y-axis in log scale

        layout.addWidget(self.plot_widget)

        # Add crosshair (cursor) for the plots
        self.v_line = pg.InfiniteLine(angle=90, movable=False)
        self.h_line = pg.InfiniteLine(angle=0, movable=False)
        self.iv_plot.addItem(self.v_line, ignoreBounds=True)
        self.iv_plot.addItem(self.h_line, ignoreBounds=True)
        
        # Button to export to CSV
        self.export_button = QPushButton('Export to CSV')
        self.export_button.clicked.connect(self.export_to_csv)
        layout.addWidget(self.export_button)

        self.setLayout(layout)

    def start_measurement(self):
        self.voltage_min = self.voltage_min_input.value()
        self.voltage_max = self.voltage_max_input.value()
        self.voltage_step = self.voltage_step_input.value()
        self.compliance_current = self.compliance_input.value()
        self.collection_time = self.collection_time_input.value()

        # Clear previous measurements
        self.measurements = []
        self.current_voltage = self.voltage_min
        self.timer.start(100)  # Update every 100 ms
        self.elapsed_timer.start()  # Start the elapsed time for integration

        # Clear plots
        self.iv_plot.clear()
        self.abs_iv_plot.clear()

    def stop_measurement(self):
        self.timer.stop()

    def perform_measurement(self):
        # Perform signal integration over the collection time
        start_time = self.elapsed_timer.elapsed()
        total_current = 0
        num_measurements = 0

        while self.elapsed_timer.elapsed() - start_time < self.collection_time:
            # Set the voltage and get the current multiple times to average
            self.simulated_device.set_voltage(self.current_voltage)
            current = self.simulated_device.measure_current()
            total_current += current
            num_measurements += 1

        # Calculate the average current during the collection time
        if num_measurements > 0:
            average_current = total_current / num_measurements
        else:
            average_current = 0

        # Check compliance current
        if abs(average_current) >= self.compliance_current:
            self.simulated_device.set_voltage(0)  # Set voltage to 0 if compliance exceeded
            self.timer.stop()

        # Store the data (voltage, average current)
        self.measurements.append((self.current_voltage, average_current))

        # Update plots
        self.update_plots()

        # Move to the next voltage step
        self.current_voltage += self.voltage_step
        if self.current_voltage > self.voltage_max:
            self.timer.stop()

    def update_plots(self):
        # Get I(V) and abs(I(V)) data
        voltages = [m[0] for m in self.measurements]
        currents = [m[1] for m in self.measurements]
        abs_currents = [abs(i) for i in currents]

        # Update I(V) plot
        self.iv_plot.plot(voltages, currents, pen=pg.mkPen(color='b', width=2), clear=True)
        self.iv_plot.setLabel('left', 'Current (A)')
        self.iv_plot.setLabel('bottom', 'Voltage (V)')

        # Update |I(V)| plot
        self.abs_iv_plot.plot(voltages, abs_currents, pen=pg.mkPen(color='r', width=2), clear=True)
        self.abs_iv_plot.setLabel('left', '|Current| (A)')
        self.abs_iv_plot.setLabel('bottom', 'Voltage (V)')

    def export_to_csv(self):
        # Open file dialog to save CSV
        file_name, _ = QFileDialog.getSaveFileName(self, 'Save CSV', '', 'CSV Files (*.csv)')
        if file_name:
            with open(file_name, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(['Voltage (V)', 'Current (A)'])
                csvwriter.writerows(self.measurements)

# Main application loop
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MeasurementSimulator()
    window.show()
    sys.exit(app.exec_())


2024-10-04 16:51:14.063 Python[52892:5792899] +[IMKClient subclass]: chose IMKClient_Modern
2024-10-04 16:51:14.063 Python[52892:5792899] +[IMKInputSession subclass]: chose IMKInputSession_Modern


SystemExit: 0

/Users/colio/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
%pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 11.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import numpy as np
import csv
import pyqtgraph as pg
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QDoubleSpinBox, QPushButton, QFileDialog)
from PyQt5.QtCore import QTimer
from scipy.fft import fft

class MeasurementSimulator(QWidget):
    def __init__(self):
        super().__init__()
        
        # Initialize simulation variables
        self.simulated_device = SimulatedKeithley6517B(mode='diode', resistance=1e3)
        self.voltage_min = 0
        self.voltage_max = 1
        self.compliance_current = 1e-3
        self.voltage_step = 0.1
        self.measurements = []
        
        # Setup GUI components
        self.initUI()
        
        # Setup timer for measurements
        self.timer = QTimer()
        self.timer.timeout.connect(self.perform_measurement)
        
    def initUI(self):
        self.setWindowTitle('Simulated Keithley 6517B')
        layout = QVBoxLayout()
        
        # Voltage range input
        voltage_range_layout = QHBoxLayout()
        self.voltage_min_input = QDoubleSpinBox()
        self.voltage_min_input.setRange(-10, 10)
        self.voltage_min_input.setValue(0)
        self.voltage_max_input = QDoubleSpinBox()
        self.voltage_max_input.setRange(-10, 10)
        self.voltage_max_input.setValue(1)
        voltage_range_layout.addWidget(QLabel('Vmin (V):'))
        voltage_range_layout.addWidget(self.voltage_min_input)
        voltage_range_layout.addWidget(QLabel('Vmax (V):'))
        voltage_range_layout.addWidget(self.voltage_max_input)
        layout.addLayout(voltage_range_layout)
        
        # Compliance current input
        compliance_layout = QHBoxLayout()
        self.compliance_input = QDoubleSpinBox()
        self.compliance_input.setRange(1e-9, 1)
        self.compliance_input.setValue(1e-3)
        compliance_layout.addWidget(QLabel('Compliance current (A):'))
        compliance_layout.addWidget(self.compliance_input)
        layout.addLayout(compliance_layout)
        
        # Start/Stop button
        button_layout = QHBoxLayout()
        self.start_button = QPushButton('Start Measurement')
        self.start_button.clicked.connect(self.start_measurement)
        self.stop_button = QPushButton('Stop Measurement')
        self.stop_button.clicked.connect(self.stop_measurement)
        button_layout.addWidget(self.start_button)
        button_layout.addWidget(self.stop_button)
        layout.addLayout(button_layout)
        
        # Plot area for I(V) and abs(I(V)) using PyQtGraph
        self.plot_widget = pg.GraphicsLayoutWidget()
        self.iv_plot = self.plot_widget.addPlot(title="I(V)")
        self.abs_iv_plot = self.plot_widget.addPlot(title="|I(V)|")
        layout.addWidget(self.plot_widget)
        
        # Button to export to CSV
        self.export_button = QPushButton('Export to CSV')
        self.export_button.clicked.connect(self.export_to_csv)
        layout.addWidget(self.export_button)
        
        self.setLayout(layout)
        
    def start_measurement(self):
        self.voltage_min = self.voltage_min_input.value()
        self.voltage_max = self.voltage_max_input.value()
        self.compliance_current = self.compliance_input.value()
        self.voltage_step = 0.1  # Voltage step for simulation
        
        # Clear previous measurements
        self.measurements = []
        self.current_voltage = self.voltage_min
        self.timer.start(100)  # Take a measurement every 100 ms
        
        # Clear plots
        self.iv_plot.clear()
        self.abs_iv_plot.clear()
        
    def stop_measurement(self):
        self.timer.stop()
        
    def perform_measurement(self):
        # Set the voltage and get the current
        self.simulated_device.set_voltage(self.current_voltage)
        current = self.simulated_device.measure_current()
        
        # Check compliance current
        if abs(current) >= self.compliance_current:
            self.simulated_device.set_voltage(0)  # Set voltage to 0 if compliance exceeded
            self.timer.stop()
        
        # Store the data (voltage, current)
        self.measurements.append((self.current_voltage, current))
        
        # Update plots
        self.update_plots()
        
        # Move to the next voltage step
        self.current_voltage += self.voltage_step
        if self.current_voltage > self.voltage_max:
            self.timer.stop()
        
    def update_plots(self):
        # Get I(V) and abs(I(V)) data
        voltages = [m[0] for m in self.measurements]
        currents = [m[1] for m in self.measurements]
        abs_currents = [abs(i) for i in currents]
        
        # Update I(V) plot
        self.iv_plot.plot(voltages, currents, pen=pg.mkPen(color='b', width=2), clear=True)
        self.iv_plot.setLabel('left', 'Current (A)')
        self.iv_plot.setLabel('bottom', 'Voltage (V)')
        
        # Update |I(V)| plot
        self.abs_iv_plot.plot(voltages, abs_currents, pen=pg.mkPen(color='r', width=2), clear=True)
        self.abs_iv_plot.setLabel('left', '|Current| (A)')
        self.abs_iv_plot.setLabel('bottom', 'Voltage (V)')
        
    def export_to_csv(self):
        # Open file dialog to save CSV
        file_name, _ = QFileDialog.getSaveFileName(self, 'Save CSV', '', 'CSV Files (*.csv)')
        if file_name:
            with open(file_name, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(['Voltage (V)', 'Current (A)'])
                csvwriter.writerows(self.measurements)

# Main application loop
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MeasurementSimulator()
    window.show()
    sys.exit(app.exec_())


: 

In [12]:
#pyvisa-py
import pymeasure
import pymeasure.instruments as pinstr
from pymeasure.instruments.resources import list_resources

from pymeasure.log import console_log
from pymeasure.display import Plotter
from pymeasure.experiment import Procedure, Results, Worker
from pymeasure.experiment import IntegerParameter, FloatParameter, Parameter
import pymeasure.instruments.keithley
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import sys
# import numpy as np
# import csv
# from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QLineEdit, QPushButton, 
#                              QSpinBox, QDoubleSpinBox, QFileDialog)
# from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
# from matplotlib.figure import Figure
# import matplotlib.pyplot as plt
# from PyQt5.QtCore import QTimer

# class MeasurementSimulator(QWidget):
#     def __init__(self):
#         super().__init__()
        
#         # Initialize simulation variables
#         self.simulated_device = SimulatedKeithley6517B(mode='diode', resistance=1e3)
#         self.voltage_min = 0
#         self.voltage_max = 1
#         self.compliance_current = 1e-3
#         self.voltage_step = 0.1
#         self.measurements = []
        
#         # Setup GUI components
#         self.initUI()
        
#         # Setup timer for measurements
#         self.timer = QTimer()
#         self.timer.timeout.connect(self.perform_measurement)
        
#     def initUI(self):
#         self.setWindowTitle('Simulated Keithley 6517B')
#         layout = QVBoxLayout()
        
#         # Voltage range input
#         voltage_range_layout = QHBoxLayout()
#         self.voltage_min_input = QDoubleSpinBox()
#         self.voltage_min_input.setRange(-10, 10)
#         self.voltage_min_input.setValue(0)
#         self.voltage_max_input = QDoubleSpinBox()
#         self.voltage_max_input.setRange(-10, 10)
#         self.voltage_max_input.setValue(1)
#         voltage_range_layout.addWidget(QLabel('Vmin (V):'))
#         voltage_range_layout.addWidget(self.voltage_min_input)
#         voltage_range_layout.addWidget(QLabel('Vmax (V):'))
#         voltage_range_layout.addWidget(self.voltage_max_input)
#         layout.addLayout(voltage_range_layout)
        
#         # Compliance current input
#         compliance_layout = QHBoxLayout()
#         self.compliance_input = QDoubleSpinBox()
#         self.compliance_input.setRange(1e-9, 1)
#         self.compliance_input.setValue(1e-3)
#         compliance_layout.addWidget(QLabel('Compliance current (A):'))
#         compliance_layout.addWidget(self.compliance_input)
#         layout.addLayout(compliance_layout)
        
#         # Start/Stop button
#         button_layout = QHBoxLayout()
#         self.start_button = QPushButton('Start Measurement')
#         self.start_button.clicked.connect(self.start_measurement)
#         self.stop_button = QPushButton('Stop Measurement')
#         self.stop_button.clicked.connect(self.stop_measurement)
#         button_layout.addWidget(self.start_button)
#         button_layout.addWidget(self.stop_button)
#         layout.addLayout(button_layout)
        
#         # Plot area for I(V) and abs(I(V))
#         self.figure = Figure()
#         self.canvas = FigureCanvas(self.figure)
#         self.ax_iv = self.figure.add_subplot(211)
#         self.ax_abs_iv = self.figure.add_subplot(212)
#         layout.addWidget(self.canvas)
        
#         # Button to export to CSV
#         self.export_button = QPushButton('Export to CSV')
#         self.export_button.clicked.connect(self.export_to_csv)
#         layout.addWidget(self.export_button)
        
#         self.setLayout(layout)
        
#     def start_measurement(self):
#         self.voltage_min = self.voltage_min_input.value()
#         self.voltage_max = self.voltage_max_input.value()
#         self.compliance_current = self.compliance_input.value()
#         self.voltage_step = 0.1  # Voltage step for simulation
        
#         # Clear previous measurements
#         self.measurements = []
#         self.current_voltage = self.voltage_min
#         self.timer.start(100)  # Take a measurement every 100 ms
        
#     def stop_measurement(self):
#         self.timer.stop()
        
#     def perform_measurement(self):
#         # Set the voltage and get the current
#         self.simulated_device.set_voltage(self.current_voltage)
#         current = self.simulated_device.measure_current()
        
#         # Check compliance current
#         if abs(current) >= self.compliance_current:
#             self.simulated_device.set_voltage(0)  # Set voltage to 0 if compliance exceeded
#             self.timer.stop()
        
#         # Store the data (voltage, current)
#         self.measurements.append((self.current_voltage, current))
        
#         # Update plots
#         self.update_plots()
        
#         # Move to the next voltage step
#         self.current_voltage += self.voltage_step
#         if self.current_voltage > self.voltage_max:
#             self.timer.stop()
        
#     def update_plots(self):
#         # Get I(V) and abs(I(V)) data
#         voltages = [m[0] for m in self.measurements]
#         currents = [m[1] for m in self.measurements]
#         abs_currents = [abs(i) for i in currents]
        
#         # Update I(V) plot
#         self.ax_iv.clear()
#         self.ax_iv.plot(voltages, currents, label='I(V)')
#         self.ax_iv.set_xlabel('Voltage (V)')
#         self.ax_iv.set_ylabel('Current (A)')
#         self.ax_iv.legend()
        
#         # Update |I(V)| plot
#         self.ax_abs_iv.clear()
#         self.ax_abs_iv.plot(voltages, abs_currents, label='|I(V)|')
#         self.ax_abs_iv.set_xlabel('Voltage (V)')
#         self.ax_abs_iv.set_ylabel('Current (A)')
#         self.ax_abs_iv.legend()
        
#         # Refresh canvas
#         self.canvas.draw()
        
#     def export_to_csv(self):
#         # Open file dialog to save CSV
#         file_name, _ = QFileDialog.getSaveFileName(self, 'Save CSV', '', 'CSV Files (*.csv)')
#         if file_name:
#             with open(file_name, 'w', newline='') as csvfile:
#                 csvwriter = csv.writer(csvfile)
#                 csvwriter.writerow(['Voltage (V)', 'Current (A)'])
#                 csvwriter.writerows(self.measurements)

# # Main application loop
# if __name__ == '__main__':
#     app = QApplication(sys.argv)
#     window = MeasurementSimulator()
#     window.show()
#     sys.exit(app.exec_())


SystemExit: 0

/Users/colio/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
list_resources()

OSError: Could not open VISA library:
Error while accessing /Library/Frameworks/visa.framework/visa: dlopen(/Library/Frameworks/visa.framework/visa, 0x0006): tried: '/Library/Frameworks/visa.framework/visa' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/visa.framework/visa' (no such file), '/Library/Frameworks/visa.framework/visa' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Library/Frameworks/visa.framework/visa' (no such file, not in dyld cache), '/Library/Frameworks/VISA.framework/Versions/A/VISA' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/VISA.framework/Versions/A/VISA' (no such file), '/Library/Frameworks/VISA.framework/Versions/A/VISA' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [15]:
keithley = pymeasure.instruments.keithley.Keithley6517B('GPIB::0')

OSError: Could not open VISA library:
Error while accessing /Library/Frameworks/visa.framework/visa: dlopen(/Library/Frameworks/visa.framework/visa, 0x0006): tried: '/Library/Frameworks/visa.framework/visa' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/visa.framework/visa' (no such file), '/Library/Frameworks/visa.framework/visa' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Library/Frameworks/visa.framework/visa' (no such file, not in dyld cache), '/Library/Frameworks/VISA.framework/Versions/A/VISA' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/VISA.framework/Versions/A/VISA' (no such file), '/Library/Frameworks/VISA.framework/Versions/A/VISA' (fat file, but missing compatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [10]:
def measure_IV(device, vmin = 0.0, vmax = 1.0, npoints = 1000):
    device.apply_voltage(voltage_range=5)
    device.source_voltage = 0
    device.enable_source()

    for v_val in np.linspace(vmin, vmax, npoints):
        device.ramp_to_voltage(v_val)
        yield (device.measure_voltage(), device.measure_resistance())
    device.disable_source()

In [8]:
#make v-i plot
import time 
def gen(n=10):
    for i in range(n):
        time.sleep(1.5)
        yield n

for i in gen(10):
    print(i)

for i in gen(7):
    print(i)

10
10
10
10
10
10
10
10
10
10
7
7
7
7
7
7
7
